In [ ]:
# This notebook needs to work with the prosculpt env.
#
# First set the conda executable of your VScode to /home/folivieri/miniforge3/etc/profile.d/conda.sh (I'm not entirely sure this is necessary)
# Then ctrl-shift-P, Select interpreter, choose path, and input /home/folivieri/.conda/envs/prosculpt/bin/python3.12

In [1]:
import pandas as pd
import os
import shutil
import glob
import pathlib
import seaborn as sns

In [ ]:
# Set requirements for filtering

plddt_threshold = 70
plddt_sculpted_threshold = 80
rmsd_threshold = 3

rmsd_fixedchains_threshold = 8
rmsd_motif_threshold = 2
rmsd_sculpted_threshold = 4

folder = "/home/d12-studenti/Minimal_Prosculpt_scripts/Project_name/output"  # Set to output folder (the one with all the sub-task folders).
folders = glob.glob(folder + "/*")
print(folders)

['/home/d12-studenti/Marc/GLP1R/Day3-Fixed/05', '/home/d12-studenti/Marc/GLP1R/Day3-Fixed/01', '/home/d12-studenti/Marc/GLP1R/Day3-Fixed/02', '/home/d12-studenti/Marc/GLP1R/Day3-Fixed/03', '/home/d12-studenti/Marc/GLP1R/Day3-Fixed/04']


In [ ]:
# Select best pdbs based on different criteria

good_results_dict = {}
copy_files = True  # Copy the files? Usually this is what you want

for subfolder in folders:
    # print(folder)
    if os.path.exists(os.path.join(subfolder, "best_pdbs")):
        shutil.rmtree(os.path.join(subfolder, "best_pdbs"))
    if os.path.exists(os.path.join(subfolder, "final_output_best.csv")):
        os.remove(os.path.join(subfolder, "final_output_best.csv"))

    if os.path.exists(os.path.join(subfolder, "final_output.csv")):
        df = pd.read_csv(os.path.join(subfolder, "final_output.csv"))
        # print(df['RMSD_sculpted'])
        # df['RMSD_sculpted'] = pd.to_numeric(df['RMSD_sculpted'])
        # best_df=df[(df["RMSD_sculpted"] <= rmsd_sculpted_threshold) & (df["plddt"] >= plddt_threshold)] #select best based on thresholds
        best_df = df[
            (df["RMSD"] <= rmsd_threshold)
            & (df["RMSD_sculpted"] <= rmsd_sculpted_threshold)
            & (df["plddt"] >= plddt_threshold)
            & (df["plddt_sculpted"] >= plddt_sculpted_threshold)
        ]  # select best based on thresholds
        # best_df=df[(df["RMSD_sculpted"] <= rmsd_sculpted_threshold) & (df["plddt_sculpted"] >= plddt_sculpted_threshold)] #select best based on thresholds
        # best_df=df[(df["RMSD_sculpted"] <= rmsd_sculpted_threshold) & (df["plddt"] >= plddt_threshold)& (df["plddt_sculpted"] >= plddt_sculpted_threshold)] #select best based on thresholds
        # best_df=df[(df["RMSD_motif"] <= rmsd_motif_threshold) & (df["plddt_sculpted"] >= plddt_sculpted_threshold) & (df["RMSD_fixed_chains"] <= rmsd_fixedchains_threshold) & (df["RMSD_sculpted"] <= rmsd_sculpted_threshold)]

        # best_df['id']= df.apply(lambda row: f"{pathlib.Path(row['path_rfdiff']).stem[1:]}.{row['link_lenght']:.{0}f}.{row['plddt_sculpted']:.{0}f}.{row['RMSD_sculpted']:.{0}f}.{row['pae']:.{0}f}", axis=1)
        # duplicates = best_df['id'].duplicated()
        # print('Number of duplicated ids: ', sum(duplicates))
        # print(best_df)

        # best_df=df[(df["plddt"] >= plddt_threshold)] #Only plddt
        # best_df=df[(df["RMSD_sculpted"] <= rmsd_sculpted_threshold)] #Only rmsd sculpted
        # best_df=df[(df["pae"] <= 8)] #Only rmsd sculpted
        dir_best_pdbs = os.path.join(subfolder, "best_pdbs")

        if copy_files:
            os.makedirs(
                dir_best_pdbs, exist_ok=True
            )  # directory is created even if some or all of the intermediate directories in the path do not exist
            best_folder = os.path.join(subfolder, "best_pdbs")
            for index, row in best_df.iterrows():
                file = str(row["model_path"])
                shutil.copy(file, os.path.join(best_folder, pathlib.Path(file).name))

                # file=file.replace('/home/folivieri/prosculpt/Fibers_Rollers_rigid_fusion/output/','/home/folivieri/prosculpt/Fibers_Rollers_rigid_fusion/99_2024-04-22/')
                # id=row["id"]
                # id_filename=id+'.pdb'
                # shutil.copy(file,os.path.join(best_folder,str(id_filename)))
        best_df = best_df.replace("final_pdbs", "best_pdbs", regex=True)

        best_df.to_csv(
            f'{os.path.join(subfolder, "final_output_best.csv")}', index=False
        )
        good_results_dict[pathlib.Path(subfolder).name] = len(best_df)

        print(subfolder, len(best_df))

/home/d12-studenti/Marc/GLP1R/Day3-Fixed/05 8
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/01 6
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/04 10


In [ ]:
# Display number of results per task
good_results_df = pd.DataFrame.from_dict(good_results_dict, "index")
pd.set_option("display.max_rows", None)
display(good_results_df.sort_values(0, ascending=False))

,0
04,10
05,8
01,6


In [ ]:
# Pool all pdbs that were selected above for each task into one folder

pdbs = glob.glob(folder + "/*/best_pdbs/*")
print(pdbs)

pooled_folder = os.path.join(folder, "pooled_best_pdbs")
os.makedirs(pooled_folder, exist_ok=True)

for pdb in pdbs:
    shutil.copy(pdb, pooled_folder)

best_csvs = glob.glob(folder + "/*/final_output_best.csv")

df_list = []
for csv_file in best_csvs:
    df = pd.read_csv(csv_file)
    df_list.append(df)
big_df = pd.concat(df_list, ignore_index=True)


def get_id_from_path(model_path):
    return os.path.basename(model_path)


big_df["file_name"] = big_df["model_path"].apply(get_id_from_path)
big_df.to_csv(os.path.join(pooled_folder, "prosculpt_metrics.csv"), index=False)

['/home/d12-studenti/Marc/GLP1R/Day3-Fixed/05/best_pdbs/5.53.1.1__link_74__plddt_88__plddt_sculpted_88__rmsd_2.0__rmsd_sculpted_3.5__rmsd_fixedchains_0.7__rmsd_motif_-1__pae_5.78__out_05_.pdb', '/home/d12-studenti/Marc/GLP1R/Day3-Fixed/05/best_pdbs/5.0.4.1__link_61__plddt_87__plddt_sculpted_86__rmsd_2.0__rmsd_sculpted_4.0__rmsd_fixedchains_0.9__rmsd_motif_-1__pae_7.81__out_05_.pdb', '/home/d12-studenti/Marc/GLP1R/Day3-Fixed/05/best_pdbs/5.53.2.1__link_74__plddt_86__plddt_sculpted_84__rmsd_1.9__rmsd_sculpted_3.2__rmsd_fixedchains_0.7__rmsd_motif_-1__pae_7.3__out_05_.pdb', '/home/d12-studenti/Marc/GLP1R/Day3-Fixed/05/best_pdbs/5.35.2.1__link_74__plddt_90__plddt_sculpted_90__rmsd_1.5__rmsd_sculpted_3.8__rmsd_fixedchains_1.0__rmsd_motif_-1__pae_5.63__out_05_.pdb', '/home/d12-studenti/Marc/GLP1R/Day3-Fixed/05/best_pdbs/5.47.4.1__link_67__plddt_87__plddt_sculpted_88__rmsd_1.8__rmsd_sculpted_3.5__rmsd_fixedchains_1.1__rmsd_motif_-1__pae_6.87__out_05_.pdb', '/home/d12-studenti/Marc/GLP1R/Day3-

In [6]:
#Run rosetta metrics on pooled best structures



input_folders=[pooled_folder] 
for folder in input_folders:   
    print(folder)
    if len(glob.glob(folder+"/*.pdb")) != 0:
        print(f"found {len(glob.glob(folder+"/*.pdb"))} pdbs")
        !sbatch --array=1-10 /home/folivieri/prosculpt_projects/ZZ_my_scripts/rosetta_metrics_jug_with_split.slurm {folder} rosetta.csv 



/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs
found 24 pdbs
Submitted batch job 721029


In [ ]:
# remove the jug data (this is important. it interferes next time you want to run it)
# shutil.rmtree("/home/folivieri/prosculpt_projects/ZZ_my_scripts/rosetta_metrics_with_split.jugdata")

In [ ]:
# Generate a Fasta

from Bio import SeqIO

pdb_list = glob.glob(pooled_folder + "/*.pdb")

with open(os.path.join(pooled_folder, "sequences.fasta"), "w+") as f:
    f.write("# --amber\n")
    for pdb in pdb_list:
        f.write(f">{pdb}\n")
        print(pdb)
        seq = ""
        for record in SeqIO.parse(os.path.join(pooled_folder, pdb), "pdb-atom"):
            seq += str(record.seq)
            seq += ":"

        seq = seq[:-1]  # delete last ":"
        seq += "\n"
        f.write(seq)


chains = [0]
with open(os.path.join(pooled_folder, "sequences_monomer.fasta"), "w+") as f:
    f.write("# --amber\n")
    for pdb in pdb_list:
        f.write(f">{pdb}\n")
        print(pdb)
        seq = ""
        chain_n = 0
        for record in SeqIO.parse(os.path.join(pooled_folder, pdb), "pdb-atom"):
            if chain_n in chains:
                seq += str(record.seq)
                seq += ":"
            chain_n += 1

        seq = seq[:-1]  # delete last ":"
        seq += "\n"
        f.write(seq)

/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/5.53.1.1__link_74__plddt_88__plddt_sculpted_88__rmsd_2.0__rmsd_sculpted_3.5__rmsd_fixedchains_0.7__rmsd_motif_-1__pae_5.78__out_05_.pdb
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/1.36.2.1__link_79__plddt_85__plddt_sculpted_83__rmsd_1.4__rmsd_sculpted_3.3__rmsd_fixedchains_0.9__rmsd_motif_-1__pae_6.94__out_01_.pdb
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/5.0.4.1__link_61__plddt_87__plddt_sculpted_86__rmsd_2.0__rmsd_sculpted_4.0__rmsd_fixedchains_0.9__rmsd_motif_-1__pae_7.81__out_05_.pdb
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/5.53.2.1__link_74__plddt_86__plddt_sculpted_84__rmsd_1.9__rmsd_sculpted_3.2__rmsd_fixedchains_0.7__rmsd_motif_-1__pae_7.3__out_05_.pdb
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/1.36.1.1__link_79__plddt_90__plddt_sculpted_90__rmsd_0.9__rmsd_sculpted_1.0__rmsd_fixedchains_0.9__rmsd_motif_-1__pae_5.91__out_01_.pdb
/home/d12-studenti/Marc/GLP1R/Da

/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: Biop

/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/5.47.4.1__link_67__plddt_87__plddt_sculpted_88__rmsd_1.8__rmsd_sculpted_3.5__rmsd_fixedchains_1.1__rmsd_motif_-1__pae_6.87__out_05_.pdb
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/1.52.5.1__link_72__plddt_92__plddt_sculpted_95__rmsd_1.2__rmsd_sculpted_2.2__rmsd_fixedchains_1.0__rmsd_motif_-1__pae_5.01__out_01_.pdb
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/5.72.4.1__link_66__plddt_91__plddt_sculpted_95__rmsd_1.8__rmsd_sculpted_3.7__rmsd_fixedchains_1.5__rmsd_motif_-1__pae_5.02__out_05_.pdb
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/5.38.2.1__link_81__plddt_94__plddt_sculpted_96__rmsd_1.4__rmsd_sculpted_2.7__rmsd_fixedchains_0.9__rmsd_motif_-1__pae_4.98__out_05_.pdb
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/4.66.5.1__link_83__plddt_88__plddt_sculpted_87__rmsd_1.6__rmsd_sculpted_3.0__rmsd_fixedchains_0.9__rmsd_motif_-1__pae_6.08__out_04_.pdb
/home/d12-studenti/Marc/GLP1R/

/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: Biop

/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/1.52.3.1__link_72__plddt_94__plddt_sculpted_97__rmsd_1.5__rmsd_sculpted_3.1__rmsd_fixedchains_1.0__rmsd_motif_-1__pae_4.59__out_01_.pdb
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/4.41.4.1__link_82__plddt_83__plddt_sculpted_85__rmsd_1.4__rmsd_sculpted_2.5__rmsd_fixedchains_0.9__rmsd_motif_-1__pae_10.19__out_04_.pdb
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/5.53.1.1__link_74__plddt_88__plddt_sculpted_88__rmsd_2.0__rmsd_sculpted_3.5__rmsd_fixedchains_0.7__rmsd_motif_-1__pae_5.78__out_05_.pdb
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/1.36.2.1__link_79__plddt_85__plddt_sculpted_83__rmsd_1.4__rmsd_sculpted_3.3__rmsd_fixedchains_0.9__rmsd_motif_-1__pae_6.94__out_01_.pdb
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/5.0.4.1__link_61__plddt_87__plddt_sculpted_86__rmsd_2.0__rmsd_sculpted_4.0__rmsd_fixedchains_0.9__rmsd_motif_-1__pae_7.81__out_05_.pdb
/home/d12-studenti/Marc/GLP1R/

/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: Biop

/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/4.16.5.1__link_84__plddt_86__plddt_sculpted_85__rmsd_1.4__rmsd_sculpted_2.6__rmsd_fixedchains_0.8__rmsd_motif_-1__pae_6.52__out_04_.pdb
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/5.35.2.1__link_74__plddt_90__plddt_sculpted_90__rmsd_1.5__rmsd_sculpted_3.8__rmsd_fixedchains_1.0__rmsd_motif_-1__pae_5.63__out_05_.pdb
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/4.2.5.1__link_83__plddt_91__plddt_sculpted_92__rmsd_1.6__rmsd_sculpted_3.4__rmsd_fixedchains_1.0__rmsd_motif_-1__pae_4.95__out_04_.pdb
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/5.47.4.1__link_67__plddt_87__plddt_sculpted_88__rmsd_1.8__rmsd_sculpted_3.5__rmsd_fixedchains_1.1__rmsd_motif_-1__pae_6.87__out_05_.pdb
/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/1.52.5.1__link_72__plddt_92__plddt_sculpted_95__rmsd_1.2__rmsd_sculpted_2.2__rmsd_fixedchains_1.0__rmsd_motif_-1__pae_5.01__out_01_.pdb
/home/d12-studenti/Marc/GLP1R/D

/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/home/folivieri/.conda/envs/prosculpt/lib/python3.12/site-packages/Bio/SeqIO/PdbIO.py:322: Biop

In [ ]:
# pooled_folder=os.path.join(folder,"pooled_best_pdbs")
pooled_folder = os.path.join(folder)
prosculpt_csv_path = os.path.join(pooled_folder, "prosculpt_metrics.csv")
rosetta_csv_path = os.path.join(pooled_folder, "rosetta.csv")


def get_monomer_path(af2_path):
    p = pathlib.Path(af2_path)
    pm = p.parent / "monomers" / ("monomer_" + p.name)
    # pm.exists()
    return str(pm)


df = pd.read_csv(rosetta_csv_path, index_col="ID")
df_prosculpt = pd.read_csv(prosculpt_csv_path)
filtered_path = os.path.join(pooled_folder, "filtered")
# df.drop(columns=['Unnamed: 0'], inplace=True)

df["file_name"] = df.index

df = pd.merge(df, df_prosculpt, how="outer", on=["file_name"])
new_order = ["file_name"] + [col for col in df.columns if col != "file_name"]
df = df[new_order]


def get_new_path(filtered_path, file_name):
    return os.path.join(filtered_path, file_name)


df["model_path"] = df.apply(
    lambda x: get_new_path(filtered_path, x["file_name"]), axis=1
)

print(df)
interesting_cols = [
    "file_name",
    "plddt",
    "plddt_sculpted",
    "pae",
    "chargeA",
    "sap_score",
    "ddg",
    "sc2",
    "vbuns",
    "sbuns",
    "RMSD",
    "RMSD_fixed_chains",
    "RMSD_motif",
]

if pathlib.Path(filtered_path).exists() and pathlib.Path(filtered_path).is_dir():
    shutil.rmtree(pathlib.Path(filtered_path))
os.makedirs(filtered_path, exist_ok=True)

dfq = df.query("plddt>70").sort_values(by="plddt", ascending=False)
# dfq = df.query('plddt_sculpted>80').sort_values(by='plddt_sculpted', ascending=False)
dfq = dfq.query("ddg<-46").sort_values(by="ddg", ascending=False)
# dfq = dfq.query('sc2>0.595').sort_values(by='sc2', ascending=False)
# dfq = dfq.query('vbuns<10').sort_values(by='vbuns', ascending=False)
# dfq = dfq.query('chargeA<-2').sort_values(by='chargeA', ascending=False)
dfq = dfq.query("RMSD_motif<=5").sort_values(by="RMSD_motif", ascending=False)

# dfq = dfq.query('ddg>-70').sort_values(by='ddg', ascending=False)

print(dfq[interesting_cols])

for index, row in dfq.iterrows():
    shutil.copy(
        os.path.join(pooled_folder, "minimized", row["file_name"]), filtered_path
    )

dfq.to_csv(os.path.join(filtered_path, "rosetta_metrics_selected.csv"))
dfs = dfq[["plddt_sculpted", "sc2", "ddg", "RMSD_motif"]]
sns.pairplot(dfs)

FileNotFoundError: [Errno 2] No such file or directory: '/home/d12-studenti/Marc/GLP1R/Day3-Fixed/pooled_best_pdbs/rosetta.csv'